In [2]:
from IPython.core.display import HTML
def css_styling():
    styles = open("custom.css", "r").read()
    return HTML(styles)
css_styling()
%matplotlib inline

In [6]:
import numpy as np
from matplotlib import pyplot as plt
from mpl_toolkits.basemap import Basemap, shiftgrid, cm
import pandas as pd
import numpy as np
import netCDF4
from matplotlib.ticker import MultipleLocator
from scipy.interpolate import *

In [7]:
press = np.array([1000,950,900,850,700,500,400,300,250,200,150,100,70,50,30,20,10])*100
nplevs= len(press)

In [8]:
height_match = np.load('height_match_N48L60_hossaini_press.npy')            

In [25]:
years = [1,2,3,4,5,6,7,8]

def read_in_data(year):
    filename='/Users/ptg21/data/projects/TRANSCOM/N48_results/CHBr3_k'+str(year)+'.nc'
    ncfile1    = netCDF4.Dataset(filename)
    lon1        = ncfile1.variables['longitude'][:]
    lat1        = ncfile1.variables['latitude'][:] 
    times1      = ncfile1.variables['t']
    hybrid_ht1  = ncfile1.variables['hybrid_ht'][:]

    field_34063      = ncfile1.variables['field34063'][:,:,:,:] # ordonez = 62
    field_34066      = ncfile1.variables['field34066'][:,:,:,:] # liang = 66
    field_34067      = ncfile1.variables['field34067'][:,:,:,:] # ziska = 67

    nmonths, nlevels, ny1, nx1 = np.shape(field_34063)
    # return ordonez, liang, ziska + dims
    return field_34063, field_34066, field_34067, nmonths, nlevels, ny1, nx1

#field_l, field_o, field_z, nmonths, nlevels1, ny1, nx1 = read_in_data(year)

In [30]:
# get file containing heights, pressures for L85 ancil
filename='/Users/ptg21/data/projects/TRANSCOM/N48_ancil_creation/xjcira.p.nc'
ncfile2     = netCDF4.Dataset(filename)

lon2        = ncfile2.variables['longitude'][:]
lat2        = ncfile2.variables['latitude'][:]
pressure    = ncfile2.variables['p'][:]

hybrid_ht2   = ncfile2.variables['hybrid_ht'][:]
times2      = ncfile2.variables['t']
nmonths2, nheights2, ny2, nx2 = np.shape(pressure)


In [26]:
def interp_lin_val(xnew, x, y, idx):
    # x,y arrays
    # xnew is the new xvalue
    # returns ynew as a linear interpolation
    # returns a value
    dx = xnew - x[idx]
    dy_dx = (y[idx]-y[idx+1])/(x[idx]-x[idx+1]) 
    # calculate a value at thew intermediate pressure
    return y[idx]+dy_dx*dx

In [ ]:
liang_res   = np.empty([nmonths2, nplevs, ny2, nx2])
ordonez_res = np.empty([nmonths2, nplevs, ny2, nx2])
ziska_res   = np.empty([nmonths2, nplevs, ny2, nx2])
# tried this using interp1d but was factor of 3 slower!  
nyl = 0
nyu = ny2

nxl = 0
nxu = nx2

for year in years:
    print year
    field_o, field_l, field_z, nmonths, nlevels1, ny1, nx1 = read_in_data(year)

    for imonth in range (0, 12):
        print imonth
        for ilat in range(nyl, nyu):
            for ilon in range (nxl, nxu): 
                x_arr = pressure[imonth, :, ilat, ilon]
                for ilev in range(0, nplevs):
                    if height_match[imonth,ilev, ilat, ilon] != -999:
                            liang_res[imonth,  ilev, ilat, ilon] = interp_lin_val(press[ilev], 
                                                                                  x_arr, 
                                                                                  field_l[imonth,:, ilat, ilon],
                                                                                  height_match[imonth, ilev, ilat, ilon])
                            ordonez_res[imonth,ilev, ilat, ilon] =  interp_lin_val(press[ilev], 
                                                                                  x_arr, 
                                                                                  field_o[imonth,:, ilat, ilon],
                                                                                  height_match[imonth, ilev, ilat, ilon])
                            ziska_res[imonth,  ilev, ilat, ilon] = interp_lin_val(press[ilev], 
                                                                                  x_arr,
                                                                                  field_z[imonth,:, ilat, ilon],
                                                                                  height_match[imonth, ilev, ilat, ilon])

                    else:
                        liang_res[imonth,  ilev, ilat, ilon] = -999
                        ordonez_res[imonth,ilev, ilat, ilon] = -999 
                        ziska_res[imonth,  ilev, ilat, ilon] = -999 


    np.save('ordonez_res'+str(year)+'.npy',ordonez_res)
    np.save('liang_res'+str(year)+'.npy',liang_res)
    np.save('ziska_res'+str(year)+'.npy',ziska_res)